<a href="https://colab.research.google.com/github/akamrume328/tennisvision/blob/feature%2F%231/notebooks/2_model_training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google ColabでのYOLOv8モデルのトレーニング (Google Drive対応版)

このノートブックは、Google Driveをマウントし、Drive上のデータセットを使用してYOLOv8モデルをトレーニングします。  
以下の内容をカバーします：  
1. Google Driveのマウント  
2. `ultralytics`ライブラリのインストール  
3. データセットの準備とパス設定  
4. YOLOv8モデルのトレーニング  

In [ ]:
# Driveマウント → ultralyticsインストール → データ展開
from google.colab import drive; drive.mount('/content/drive', force_remount=True)
!pip install ultralytics

import os, shutil, zipfile
from tqdm import tqdm

# データセットをzipファイルから展開
print("データセットのzip展開を開始します...")
drive_zip   = '/content/drive/MyDrive/datasets/final_merged_dataset.zip'
local_data  = '/content/datasets'
if os.path.exists(local_data):
    shutil.rmtree(local_data)

with zipfile.ZipFile(drive_zip, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    print(f"展開対象ファイル数: {len(file_list)}")
    
    for file in tqdm(file_list, desc="展開中", unit="files"):
        zip_ref.extract(file, local_data)

print("データセットの展開が完了しました。")

In [ ]:
# パス設定とハイパーパラメータ
dataset_yaml_path = os.path.join(local_data, 'data_colab.yaml')

# 出力先
project_output_dir  = '/content/drive/MyDrive/models/YOLOv8_Training_Outputs'
os.makedirs(project_output_dir, exist_ok=True)

# 実験設定
experiment_name      = 'tennis_detection_drive_run'
training_device      = 'cuda:0'
checkpoint_to_resume = None
save_every_n_epochs  = 10

print(f"YAML: {dataset_yaml_path}")
print(f"出力ディレクトリ: {project_output_dir}")
print(f"実験名: {experiment_name}, デバイス: {training_device}")
print(f"チェックポイント毎: {save_every_n_epochs} epoch")

## YOLOv8モデルの初期化とトレーニング

### モデルのロードとトレーニング実行

このセルでは、YOLOv8モデルを初期化し、トレーニングを実行します。

**チェックポイントからの再開:**
前の「パスの設定」セルで `checkpoint_to_resume` に有効なチェックポイントファイル（例: `last.pt` や特定の `epoch_XXX.pt`）のパスが指定されている場合、モデルはそのチェックポイントからロードされ、トレーニングが再開されます。これにより、中断したトレーニングを続けたり、既存のモデルをさらにファインチューニングしたりできます。
`checkpoint_to_resume` が `None` であるか、指定されたパスにファイルが存在しない場合は、`model_name` で指定された新しい事前学習済みモデル（例: `yolov8s.pt`）からトレーニングが開始されます。

**定期的なチェックポイント保存:**
`model.train()` メソッドの `save_period` パラメータに、前のセルで設定した `save_every_n_epochs` の値が渡されます。
`save_every_n_epochs` が1以上の場合、指定されたエポック数ごとにモデルの重みが `project_output_dir/experiment_name/weights/epoch_XXX.pt` という名前で保存されます。これにより、トレーニングの途中経過を細かく保存し、必要に応じて特定のエポックから再開できるようになります。

**その他のパラメータ:**
-   `data`: データセット設定ファイル (`data.yaml`) へのパス。
-   `epochs`: 総トレーニングエポック数。
-   `imgsz`: 入力画像のサイズ。
-   `batch`: バッチサイズ。
-   `project`: トレーニング結果が保存される親ディレクトリ。
-   `name`: 今回のトレーニング実行固有のサブディレクトリ名。
-   `exist_ok=True`: `project/name` ディレクトリが既に存在する場合でもエラーとせず、上書きまたは追記を許可します。
-   `device`: トレーニングに使用するデバイス（例: `'cpu'`, `'cuda:0'`）。前のセルで設定した `training_device` 変数の値が使用されます。

トレーニングが開始されると、進行状況、損失、評価メトリクスなどが表示されます。
**トレーニングが正常に完了した場合、このセルの処理がすべて終了すると、Colabのランタイムは自動的に切断され、リソースが解放されます。エラーが発生した場合は、ランタイムは切断されません。**

In [ ]:
from ultralytics import YOLO
import os
from google.colab import runtime
import torch
import gc

# 改良されたGPU メモリクリア関数
def clear_gpu_memory(verbose=True):
    """GPU VRAMを完全にクリアする関数"""
    try:
        if torch.cuda.is_available():
            # 使用前のメモリ状況を表示
            if verbose:
                print(f"GPU メモリクリア前:")
                print(f"  割り当て済み: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
                print(f"  予約済み: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
            
            # キャッシュクリア
            torch.cuda.empty_cache()
            # ガベージコレクション実行
            gc.collect()
            # 再度キャッシュクリア
            torch.cuda.empty_cache()
            
            if verbose:
                print(f"GPU メモリクリア後:")
                print(f"  割り当て済み: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
                print(f"  予約済み: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
                print("GPUメモリキャッシュをクリアしました。")
        else:
            if verbose:
                print("CUDAが利用できないため、GPUメモリクリアはスキップされました。")
    except Exception as e:
        print(f"GPUメモリクリア中にエラーが発生しました: {e}")

def force_cleanup_models():
    """モデル変数を削除してメモリを解放"""
    try:
        # グローバル変数のモデルを削除
        if 'model' in globals():
            del globals()['model']
        if 'results' in globals():
            del globals()['results']
        
        # ガベージコレクション実行
        gc.collect()
        print("モデル変数を削除しました。")
    except Exception as e:
        print(f"モデルクリーンアップ中にエラー: {e}")

def show_gpu_memory_info():
    """現在のGPUメモリ使用状況を表示"""
    if torch.cuda.is_available():
        print("=== GPU メモリ使用状況 ===")
        print(f"GPU デバイス: {torch.cuda.get_device_name()}")
        print(f"総メモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        print(f"割り当て済み: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"予約済み: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
        print(f"利用可能: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1024**3:.2f} GB")
        print("========================")
    else:
        print("CUDA が利用できません。")

# 事前学習済みのYOLOv8モデルをロードするか、チェックポイントから再開
# リソース制約下でのテニスボール検出にはnanoサイズが最適
# メモリ効率が良く、適切なパラメータ調整で十分な精度を実現可能
model_name = 'yolov8n.pt' # リソース効率重視（メモリ使用量最小）

model_load_path = model_name # デフォルトは新規モデル
if checkpoint_to_resume and os.path.exists(checkpoint_to_resume):
    print(f"チェックポイントからトレーニングを再開します: {checkpoint_to_resume}")
    model_load_path = checkpoint_to_resume
elif checkpoint_to_resume:
    print(f"指定されたチェックポイント '{checkpoint_to_resume}' が見つからなかったため、新しい事前学習済みモデル ({model_name}) からトレーニングを開始します。")
else:
    print(f"新しい事前学習済みモデル ({model_name}) からトレーニングを開始します。")

model = YOLO(model_load_path)

# YOLOv8n用テニスボール検出最適化パラメータ
# nサイズでも効果的な小物体検出を実現
num_epochs = 120  # nサイズは軽量なので多めのエポック数で学習強化

# バッチサイズの設定オプション：
# 1. 固定値: 8, 16, 32 など具体的な数値を指定
# 2. 自動調整: -1 を指定すると、Ultralyticsが自動的に最適なバッチサイズを検出
#    GPUメモリに基づいて最大可能なバッチサイズが自動設定されます
batch_size = -1   # -1: 自動バッチサイズ調整 (推奨) / nサイズなら大きなバッチも可能

img_size = 1920    # 入力画像サイズ (固定)

# オプティマイザ / 学習率（nサイズ用に最適化）
optimizer       = 'AdamW'
lr0             = 3e-3      # nサイズには高めの学習率が効果的
lrf             = 0.1       # 適度な最終学習率
warmup_epochs   = 3.0       # 短めのウォームアップ（nサイズは収束が早い）
weight_decay    = 0.0005    # 軽い正則化

# nサイズ用データ拡張（軽量モデルに最適化）
mosaic          = 1.0       # 最大確率でMosaic（特徴学習強化）
mixup           = 0.1       # 軽微なMixUp（nサイズには控えめ）
augment         = True      # 基本拡張を有効化

# nサイズ用追加パラメータ（軽量化重視）
copy_paste      = 0.5       # 強めのCopy-Paste（小物体学習強化）
degrees         = 0.0       # 回転なし
translate       = 0.2       # やや大きめの平行移動（データ多様性向上）
scale           = 0.9       # スケール変動を抑制
shear           = 0.0       # せん断変形なし
perspective     = 0.0       # 透視変換なし
flipud          = 0.0       # 上下反転なし
fliplr          = 0.5       # 左右反転

# DataLoader（nサイズ用に最適化）
workers         = 2         # 軽量モデルなので少なめで十分

print(f"=== YOLOv8n テニスボール検出用パラメータ ===")
print(f"モデル: {model_name} (リソース効率重視)")
print(f"データセット: 8400フレーム")
print(f"エポック数: {num_epochs} (nサイズ用に調整)")
print(f"画像サイズ: {img_size} (高解像度固定)")
print("軽量モデル用最適化設定")
print("メモリ使用量を最小限に抑制")

print(f"\nモデルのトレーニングを開始します。")
print(f"使用デバイス: {training_device}")
if batch_size == -1:
    print(f"エポック数: {num_epochs}, バッチサイズ: 自動調整, 画像サイズ: {img_size}")
    print("nサイズモデルなので大きなバッチサイズが期待できます。")
else:
    print(f"エポック数: {num_epochs}, バッチサイズ: {batch_size}, 画像サイズ: {img_size}")
if save_every_n_epochs > 0:
    print(f"チェックポイントは {save_every_n_epochs} エポックごとに保存されます。")

print("\n重要: トレーニングが正常に完了した場合、このセルの最後にランタイムが自動的に切断されます。")
print("エラーが発生した場合は、ランタイムは切断されず、エラーメッセージを確認できます。")
print(f"トレーニング結果はマウントされたGoogle Driveディレクトリ ({project_output_dir}) に保存される設定になっています。")

# 開始前のメモリ状況確認
show_gpu_memory_info()

# トレーニングを開始
training_successful = False
try:
    print("=== トレーニング開始前のGPUメモリクリア ===")
    clear_gpu_memory()

    results = model.train(
        data            = dataset_yaml_path,
        epochs          = num_epochs,
        imgsz           = img_size,
        batch           = batch_size,  # -1 の場合は自動調整が実行される
        project         = project_output_dir,
        name            = experiment_name,
        exist_ok        = True,
        save_period     = save_every_n_epochs,
        device          = training_device,
        optimizer       = optimizer,
        lr0             = lr0,
        lrf             = lrf,
        warmup_epochs   = warmup_epochs,
        weight_decay    = weight_decay,
        mosaic          = mosaic,
        mixup           = mixup,
        augment         = augment,
        copy_paste      = copy_paste,
        degrees         = degrees,
        translate       = translate,
        scale           = scale,
        shear           = shear,
        perspective     = perspective,
        flipud          = flipud,
        fliplr          = fliplr,
        workers         = workers,
        cache           = False,  # メモリ効率化
        amp             = True,   # AMP (自動混合精度) を有効化
        patience        = 25,     # 長めのpatience（nサイズは時間がかかる場合がある）
        close_mosaic    = 15,     # 最終15エポックでMosaic無効化
    )
    print("\nトレーニングが正常に完了しました！")
    full_output_path = os.path.join(project_output_dir, experiment_name)
    print(f"結果、ログ、モデルの重みはマウントされたGoogle Drive上の次の場所に保存されました: {full_output_path}")
    training_successful = True

    # トレーニング完了後のメモリクリア
    print("\n=== トレーニング完了後のメモリクリア ===")
    force_cleanup_models()
    clear_gpu_memory()

    print("\nトレーニングが正常に完了したため、ランタイムを切断します...")
    print(f"必要なファイルがGoogle Drive ({full_output_path}) に保存されていることを確認してください。")
    print("ランタイム切断中...")
    runtime.unassign()
    print("ランタイムが切断されました。Colabの接続は解除されます。")

except torch.cuda.OutOfMemoryError as e:
    print("=== GPU OOM エラーが発生しました ===")
    print(f"エラー詳細: {e}")
    show_gpu_memory_info()
    print("\n緊急メモリクリアを実行します...")
    force_cleanup_models()
    clear_gpu_memory()
    print("\n対処法:")
    print("1. 自動バッチサイズ調整を使用: batch_size = -1 (推奨)")
    if batch_size != -1:
        print(f"2. バッチサイズを小さくする (現在: {batch_size} → 推奨: {max(1, batch_size//2)})")
    print(f"3. 画像サイズを下げる (現在: {img_size} → 推奨: 1280)")
    print("4. nサイズは最軽量なので、これ以上の軽量化は困難")
    print(f"5. workers数を減らす (現在: {workers} → 推奨: 1)")

except Exception as e:
    print(f"トレーニング中に予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()
    
    # エラー時もメモリクリア
    print("\nエラー後のメモリクリアを実行します...")
    force_cleanup_models()
    clear_gpu_memory()
finally:
    if not training_successful:
        print("\nこのセルの処理は終了しましたが、トレーニング中にエラーが発生したか、正常に完了しませんでした。")
        print(f"エラーが発生した場合でも、部分的な結果がGoogle Drive ({os.path.join(project_output_dir, experiment_name)}) に保存されている可能性があります。")
        print("\n最終的なメモリ状況:")
        show_gpu_memory_info()

## トレーニング結果と保存されたモデル

トレーニング後、結果（メトリクス、混同行列、モデルの重みなど）は指定したマウントされたGoogle Driveディレクトリに保存されます：
`{project_output_dir}/{experiment_name}` (例: `/content/drive/MyDrive/models/YOLOv8_Training_Outputs/tennis_detection_drive_run`)

確認すべき主なファイル：
- **重み:** `weights`サブディレクトリ内 (例: `best.pt`, `last.pt`)
  - `best.pt`: 最良の検証メトリクス（通常はmAP50-95）を達成したモデルの重み。このモデルを推論に使用するのが一般的です。
  - `last.pt`: トレーニングの最終エポックのモデルの重み。
  - `epoch_XXX.pt` (例: `epoch_10.pt`, `epoch_20.pt`): `save_period` で指定されたエポックごとに保存されるチェックポイント。これらを使用して特定のエポックからトレーニングを再開したり、その時点でのモデル性能を評価したりできます。
- **結果CSV:** `results.csv`にはエポックごとのメトリクスの概要が含まれています。
- **プロット:** 混同行列、P-R曲線などのさまざまなプロット (例: `confusion_matrix.png`, `PR_curve.png`)

これらのファイルは自動的にGoogle Driveに同期されるため、マウントが解除された後でもGoogle Drive上で利用できます。

In [ ]:
import os

# 実験の重みディレクトリ (マウントされたローカルパス)
weights_dir = os.path.join(project_output_dir, experiment_name, 'weights')

print(f"\n重みディレクトリ内のファイルをリストアップします ({weights_dir}):")

if os.path.exists(weights_dir):
    try:
        files_in_weights_dir = os.listdir(weights_dir)
        if files_in_weights_dir:
            for f_name in files_in_weights_dir:
                print(f"- {f_name}")
        else:
            print(f"重みディレクトリ ({weights_dir}) は空です。")
            print("トレーニングが完了していないか、エラーが発生した可能性があります。")
    except Exception as e:
        print(f"重みディレクトリの内容を読み取れませんでした: {e}")
else:
    print(f"\n重みディレクトリが見つかりません: {weights_dir}")
    print("トレーニングが完了していないか、エラーが発生した可能性があります。")

# トレーニング済みモデルの最良の重みへのローカルパス
best_model_path = os.path.join(weights_dir, 'best.pt')
print(f"\n最良のモデルへのローカルパス: {best_model_path}")

if os.path.exists(best_model_path):
    print("最良のモデルファイル (best.pt) が見つかりました。")
    # ファイルサイズも表示
    try:
        file_size = os.path.getsize(best_model_path)
        print(f"ファイルサイズ: {file_size / (1024*1024):.1f} MB")
    except Exception as e:
        print(f"ファイルサイズを取得できませんでした: {e}")
else:
    print("最良のモデルファイル (best.pt) が見つかりません。トレーニングが失敗したか、生成されなかった可能性があります。")

print(f"\n注意: ファイルは自動的にGoogle Drive (/content/drive/MyDrive) に同期されています。")

## VRAM手動開放セル

OOMエラーが発生した場合や、メモリを手動で開放したい場合に実行してください。

In [ ]:
# 手動でVRAMを完全開放するセル
print("=== 手動VRAM開放を実行します ===")

# すべてのモデル変数を削除
try:
    if 'model' in globals():
        del model
        print("モデル変数を削除しました")
    if 'results' in globals():
        del results
        print("results変数を削除しました")
except:
    pass

# ガベージコレクション
import gc
gc.collect()

# CUDA キャッシュクリア
if torch.cuda.is_available():
    # 現在のメモリ使用量表示
    print(f"クリア前 - 割り当て済み: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"クリア前 - 予約済み: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    
    # メモリクリア実行
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()  # プロセス間通信のキャッシュもクリア
    
    # 完全リセット（強制的）
    try:
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()
    except:
        pass
    
    # 再度ガベージコレクションとキャッシュクリア
    gc.collect()
    torch.cuda.empty_cache()
    
    print(f"クリア後 - 割り当て済み: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"クリア後 - 予約済み: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    print("VRAM開放完了")
else:
    print("CUDA が利用できません")

print("=== 手動VRAM開放完了 ===")

## ランタイム完全再起動

上記の方法でもメモリが解放されない場合は、以下のセルを実行してランタイムを完全に再起動してください。

In [ ]:
# ランタイムを完全に再起動（最終手段）
import os
print("ランタイムを完全に再起動します...")
print("実行後、すべてのメモリがクリアされ、セッションが終了します。")
print("再度最初のセルから実行してください。")

# Google Driveが既にマウントされていることを確認
if os.path.exists('/content/drive'):
    print("Google Drive は既にマウントされています。")
else:
    print("注意: 再起動後、Google Drive の再マウントが必要です。")

# 強制再起動
os.kill(os.getpid(), 9)